# First Test for Volatility Based VaR

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import volatility_var as vv
import backtesting as bt
import plots 
import expected_shortfall as ES

### Data

In [2]:
# Download data
sp500_data = yf.download("^GSPC", start="1980-01-01", end="2021-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### Garch based VaR on S&P 500

In [3]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [4]:
# Apply GARCH volatility model
garch_output, next_day_var = vv.var_garch(returns, confidence_level, vol_model="GJR", distribution="skewt")

print(f"Next-day GARCH VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
garch_output = ES.es_volatility(garch_output, confidence_level)

Next-day GARCH VaR estimate (abs): 1.50%


In [5]:
# Plot interactive VaR
fig_var = plots.plot_backtest(garch_output, subset=("2019-11-30", "2020-11-30"), interactive = True)

fig_var_long = plots.plot_backtest(garch_output, interactive = True)

# Plot interactive volatility for a subset
fig_vol = plots.plot_volatility(garch_output["Volatility"], subset=("1980-11-01", "2020-11-30"), interactive=True)

In [6]:
print(next_day_var)

0.01502410114029475


In [7]:
garch_output.head()

,Returns,Volatility,Innovations,VaR,VaR Violation,ES
Date,,,,,,
1980-01-03,-0.005119,0.009098,-0.562656,0.023363,False,0.032340
1980-01-04,0.012279,0.008980,1.367467,0.023059,False,0.031919
1980-01-07,0.002719,0.008791,0.309266,0.022575,False,0.031249
1980-01-08,0.019838,0.008442,2.349959,0.021677,False,0.030007
1980-01-09,0.000917,0.008608,0.106586,0.022105,False,0.030598


### Arch


In [8]:
# Apply ARCH volatility model
arch_output, next_day_var = vv.var_arch(returns, confidence_level)

print(f"Next-day ARCH VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
arch_output = ES.es_volatility(arch_output, confidence_level)

Next-day ARCH VaR estimate (abs): 2.69%


### Garch Forecasts (with Wealth Scaling)

All works as intended

In [9]:
# Configuration grid
steps_list = [3, 10, 20]
cumulative_list = [False, True]

print("GARCH(1,1) VaR Forecasts (Normal Only)\n" + "-" * 40)

for steps_ahead, cumulative in product(steps_list, cumulative_list):
    try:
        var_value = vv.garch_forecast(
            returns=returns,
            steps_ahead=steps_ahead,
            cumulative=cumulative,
            compute_var=True,
            confidence_level=0.99
        )

        label = f"VaR | {steps_ahead}-day | {'CUM' if cumulative else 'Single'}"
        print(f"{label:<35}: {var_value * 100:.4f}%")  # Convert to percent

    except Exception as e:
        print(f"{label:<35}: ERROR → {e}")


GARCH(1,1) VaR Forecasts (Normal Only)
----------------------------------------
VaR | 3-day | Single               : 1.8260%
VaR | 3-day | CUM                  : 3.0538%
VaR | 10-day | Single              : 2.0135%
VaR | 10-day | CUM                 : 5.9063%
VaR | 20-day | Single              : 2.2184%
VaR | 20-day | CUM                 : 8.9147%


In [10]:
# Test with wealth scaling for VaR
print("\nGARCH(1,1) VaR Forecasts (Scaled by Wealth = $1,000,000)\n" + "-" * 55)

for steps_ahead, cumulative in product(steps_list, cumulative_list):
    try:
        var_value = vv.garch_forecast(
            returns=returns,
            steps_ahead=steps_ahead,
            cumulative=cumulative,
            compute_var=True,
            confidence_level=0.99,
            wealth=1_000_000
        )

        label = f"VaR | {steps_ahead}-day | {'CUM' if cumulative else 'Single'}"
        print(f"{label:<45}: ${var_value:,.2f}")

    except Exception as e:
        print(f"{label:<45}: ERROR → {e}")


# Test that variance forecast with wealth raises an error
print("\nTesting error when computing variance with wealth...\n")

try:
    test = vv.garch_forecast(
        returns=returns,
        steps_ahead=10,
        cumulative=True,
        compute_var=False,  # asking for variance
        wealth=1_000_000     # should raise error
    )
except Exception as e:
    print(f"Expected error: {e}")



GARCH(1,1) VaR Forecasts (Scaled by Wealth = $1,000,000)
-------------------------------------------------------
VaR | 3-day | Single                         : $18,260.44
VaR | 3-day | CUM                            : $30,538.15
VaR | 10-day | Single                        : $20,135.23
VaR | 10-day | CUM                           : $59,063.11
VaR | 20-day | Single                        : $22,183.57
VaR | 20-day | CUM                           : $89,147.28

Testing error when computing variance with wealth...

Expected error: Wealth can only be used when compute_var=True


### MA based VaR on the same data

In [11]:
# Apply MA volatility model
ma_output, next_day_var = vv.var_moving_average(returns, confidence_level)

print(f"Next-day MA VaR estimate (abs): {100 * next_day_var:.2f}%")

# Compute ES for the whole period
ma_output = ES.es_volatility(ma_output, confidence_level)

Next-day MA VaR estimate (abs): 1.31%


In [12]:
# Plot interactive VaR
fig_var = plots.plot_backtest(ma_output, subset = ("2019-11-01", "2020-11-30"), interactive = True, )

### Different volatilities with different models

They look ok, double check later on...

In [13]:
# Apply GARCH volatility model
garch_output_classic, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="GARCH")
garch_output_grj, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="GJR")
garch_output_egarch, next_day_var_classic = vv.var_garch(returns, confidence_level, vol_model="EGARCH")
garch_output_classic_ged, next_day_var_classic_ged = vv.var_garch(returns, confidence_level, vol_model="GARCH", distribution="ged")
garch_output_classic_skewedt, next_day_var_classic_skewedt = vv.var_garch(returns, confidence_level, vol_model="GARCH", distribution="skewt")

In [14]:
# Plot interactive volatility for a subset
fig_vol_garch = plots.plot_volatility(garch_output_classic["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_gjr = plots.plot_volatility(garch_output_grj["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_egarch = plots.plot_volatility(garch_output_egarch["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_garch_ged = plots.plot_volatility(garch_output_classic_ged["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_garch_skewedt = plots.plot_volatility(garch_output_classic_skewedt["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_arch = plots.plot_volatility(arch_output["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)
fig_vol_ma = plots.plot_volatility(ma_output["Volatility"], subset=("2019-11-01", "2020-11-30"), interactive=True)


### Backtesting (Extensive)

In [15]:
# 1. Count violations
total_violations, violation_rate = bt.count_violations(garch_output)

# 2. Kupiec test
kupiec_results = bt.kupiec_test(total_violations, len(garch_output), confidence_level)

# 3. Christoffersen test
christoffersen_results = bt.christoffersen_test(garch_output)

# 4. Joint test
joint_results = bt.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

# 5. Display results
print("=== VaR Backtesting Summary: GJR-GARCH Model ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary: GJR-GARCH Model ===
Total Violations: 104
Violation Rate: 1.01%

-- Kupiec Test --
LR_uc: 0.0036282910944009927
p_value: 0.9519682332074608
reject_null: False

-- Christoffersen Test --
LR_c: 2.487951340249765
p_value: 0.11472096807344656
reject_null: False

-- Joint Test --
LR_total: 2.491579631344166
p_value: 0.2877135776752906
reject_null: False


In [16]:
# 1. Count violations
total_violations, violation_rate = bt.count_violations(ma_output)

# 2. Kupiec test
kupiec_results = bt.kupiec_test(total_violations, len(ma_output), confidence_level)

# 3. Christoffersen test
christoffersen_results = bt.christoffersen_test(ma_output)

# 4. Joint test
joint_results = bt.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

# 5. Display results
print("=== VaR Backtesting Summary: MA Model ===")
print(f"Total Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")

print("\n-- Kupiec Test --")
for k, v in kupiec_results.items():
    print(f"{k}: {v}")

print("\n-- Christoffersen Test --")
for k, v in christoffersen_results.items():
    print(f"{k}: {v}")

print("\n-- Joint Test --")
for k, v in joint_results.items():
    print(f"{k}: {v}")


=== VaR Backtesting Summary: MA Model ===
Total Violations: 104
Violation Rate: 1.01%

-- Kupiec Test --
LR_uc: 0.006248231092513379
p_value: 0.9369962311297633
reject_null: False

-- Christoffersen Test --
LR_c: 8.029196751922427
p_value: 0.0046029247841576915
reject_null: True

-- Joint Test --
LR_total: 8.03544498301494
p_value: 0.017993899566404736
reject_null: True


### Appendix

#### 1. GARCH(p, q) VaR  
Standard GARCH captures volatility clustering using past squared shocks and variances.

$$
r_t = \mu + \varepsilon_t, \quad \varepsilon_t = \sigma_t z_t
$$

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 2. EGARCH(p, q) VaR  
EGARCH models log-volatility and captures asymmetry in the volatility response.

$$
\log(\sigma_t^2) = \omega + \sum_{i=1}^q \beta_i \log(\sigma_{t-i}^2) + \sum_{j=1}^p \alpha_j \left( \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} - \mathbb{E}\left[ \frac{|\varepsilon_{t-j}|}{\sigma_{t-j}} \right] \right) + \gamma_j \frac{\varepsilon_{t-j}}{\sigma_{t-j}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 3. GJR-GARCH(p, q) VaR  
GJR-GARCH adds an indicator term to account for the leverage effect of negative shocks.

$$
\sigma_t^2 = \omega + \sum_{i=1}^q \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2 + \sum_{i=1}^q \gamma_i \varepsilon_{t-i}^2 \cdot \mathbb{I}_{\{\varepsilon_{t-i}<0\}}
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 4. APARCH(p, q) VaR  
APARCH generalizes GARCH with power terms and asymmetry.

$$
\sigma_t^\delta = \omega + \sum_{i=1}^q \alpha_i \left( |\varepsilon_{t-i}| - \gamma_i \varepsilon_{t-i} \right)^\delta + \sum_{j=1}^p \beta_j \sigma_{t-j}^\delta
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 5. ARCH(p) VaR  
ARCH models volatility using only past squared residuals.

$$
\sigma_t^2 = \omega + \sum_{i=1}^p \alpha_i \varepsilon_{t-i}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 6. EWMA VaR  
EWMA assigns exponentially decaying weights to past squared returns.

$$
\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1 - \lambda) r_{t-1}^2
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 7. MA VaR  
Moving Average volatility is estimated using a rolling window of past squared returns.

$$
\sigma_t = \sqrt{ \frac{1}{n} \sum_{i=1}^n r_{t-i}^2 }
$$

$$
\text{VaR}_t = -\hat{\sigma}_t \cdot z_\alpha
$$

#### 8. Expected Shortfall (ES)  
Expected Shortfall estimates the conditional expectation of loss beyond the VaR.

$$
\text{ES}_t = -\hat{\sigma}_t \cdot \mathbb{E}[z_t \mid z_t < z_\alpha]
$$

---

#### 9. GARCH(1,1) Variance Forecast – Analytical  

Forecast for single-step variance $T$ steps ahead:

$$
\mathbb{E}[\sigma_{t+T}^2] = \text{VL} + (\alpha + \beta)^T \cdot (\sigma_t^2 - \text{VL})
$$

Forecast for cumulative variance over $T$ days:

$$
\mathbb{E}[\sigma_{t,T}^2] =
\text{VL} \cdot \left( T - 1 - \frac{(\alpha + \beta)(1 - (\alpha + \beta)^{T - 1})}{1 - (\alpha + \beta)} \right)
+ \sigma_t^2 \cdot \frac{1 - (\alpha + \beta)^T}{1 - (\alpha + \beta)}
$$

Where:

- $\text{VL} = \dfrac{\omega}{1 - \alpha - \beta}$ is the long-run variance  
- $\sigma_t^2$ is the last fitted conditional variance  


#### 10. GARCH(1,1) VaR Forecast – Analytical  

Once variance is forecasted, the VaR over the horizon $T$ is:

$$
\text{VaR}_{t,T} = - z_\alpha \cdot \sqrt{\mathbb{E}[\sigma_{t,T}^2]}
$$

Where $z_\alpha$ is the quantile of the fitted standardized innovation distribution:

- Normal: $z_\alpha = \Phi^{-1}(1 - \alpha)$  
- Student-t / GED: fitted on standardized residuals
